# 01 - Exploratory Data Analysis
Quick exploration of job postings sourced from the scraper or a baked-in sample dataset.


In [1]:
from pathlib import Path
import sys

# run once, near the top of your notebook
repo_root = Path(__file__).resolve().parents[1] if "__file__" in globals() else Path.cwd().parents[1]
if str(repo_root) not in sys.path:
    sys.path.append(str(repo_root))


In [5]:
repo_root

WindowsPath('c:/Users/nirma/OneDrive - Lambton College/Documents/1.ML_AL_PROJECTS/Ongoing projects/Resume-Matcher')

In [2]:
from collections import Counter
from pathlib import Path

import pandas as pd

from src.storage.json_repository import JSONJobStore
from src.preprocessing.text_cleaner import TextCleaner
from src.preprocessing.skill_extractor import SkillExtractor


In [ ]:



# else:
#     sample_jobs = [
#         {
#             "job_title": "Machine Learning Engineer",
#             "job_description": "Build and deploy ML models with Python, scikit-learn, and AWS services.",
#             "company": "Acme AI",
#             "location": "Remote",
#             "posted_date": "Posted 2 days ago",
#             "source": "sample",
#             "fetched_at": pd.Timestamp.utcnow(),
#         },
#         {
#             "job_title": "Data Scientist",
#             "job_description": "Analyze datasets using Python, pandas, SQL, and visualization dashboards.",
#             "company": "DataWorks",
#             "location": "Toronto, ON",
#             "posted_date": "Posted 1 week ago",
#             "source": "sample",
#             "fetched_at": pd.Timestamp.utcnow(),
#         },
#         {
#             "job_title": "MLOps Specialist",
#             "job_description": "Manage ML pipelines with Docker, Kubernetes, and Terraform.",
#             "company": "DeployLab",
#             "location": "Vancouver, BC",
#             "posted_date": "Posted today",
#             "source": "sample",
#             "fetched_at": pd.Timestamp.utcnow(),
#         },
#     ]
#     df = pd.DataFrame(sample_jobs)

df.head()


,job_title,job_description,company,location,posted_date,source,fetched_at
0,Machine Learning Engineer,"Build and deploy ML models with Python, scikit...",Acme AI,Remote,Posted 2 days ago,sample,2025-09-22 20:46:12.402283+00:00
1,Data Scientist,"Analyze datasets using Python, pandas, SQL, an...",DataWorks,"Toronto, ON",Posted 1 week ago,sample,2025-09-22 20:46:12.404553+00:00
2,MLOps Specialist,"Manage ML pipelines with Docker, Kubernetes, a...",DeployLab,"Vancouver, BC",Posted today,sample,2025-09-22 20:46:12.404553+00:00


In [6]:
store = JSONJobStore()
records = store.recent(limit=50)

In [7]:
records

[]

In [ ]:
if records:
    df = pd.DataFrame(
        {
            "job_title": [r.job_title for r in records],
            "job_description": [r.job_description for r in records],
            "company": [r.company for r in records],
            "location": [r.location for r in records],
            "posted_date": [r.posted_date for r in records],
            "source": [r.source for r in records],
            "fetched_at": [r.fetched_at for r in records],
        }
    )

In [ ]:
df[["job_title", "company", "location", "posted_date"]]


In [ ]:
cleaner = TextCleaner(stopwords={"and", "with", "the", "to"})
df["clean_description"] = df["job_description"].fillna("").apply(cleaner.clean)
df[["job_title", "clean_description"]].head()


In [ ]:
extractor = SkillExtractor()
skill_counts: Counter[str] = Counter()
for text in df["job_description"].dropna():
    for match in extractor.extract(text):
        skill_counts[match.skill] += match.occurrences

skill_df = pd.DataFrame(
    [
        {"skill": skill, "count": count}
        for skill, count in skill_counts.most_common()
    ]
)

skill_df.head(10)
